<a href="https://colab.research.google.com/github/cgarciav/base-de-datos-HCC/blob/master/SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
#%load_ext tensorboard    Carga de Tensorboard (opcional)
# Install TensorFlow
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf #libreria de google que permite hacer deeplearning
from tensorflow import keras #libreria de alto nivel
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np
import random
import pandas as pd

In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [0]:
path='/content/drive/My Drive/analisis de datos/hcc-data-complete-balanced.csv'
df=pd.read_csv(path)

In [0]:
df

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

In [0]:
x = df.drop(['Class'], axis=1) 
y=df['Class']
x_train, x_test, y_train, y_test = train_test_split(x, y,stratify=y, test_size=0.2, random_state=42)

In [0]:
#normalizamos datos
from sklearn import preprocessing

scaler=preprocessing.StandardScaler().fit(x_train)

x_train_transformed=scaler.transform(x_train)
x_test_transformed=scaler.transform(x_test)

y_train_transformed=y_train
y_test_transformed=y_test

**Modelo basado en una Máquina de Vectores Soporte sobre el espacio original**

In [0]:
from sklearn.metrics import confusion_matrix
from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

In [0]:
c_values=np.logspace(-10,0, 10)
cv_c_scores=[]


for k in c_values:

    clf = svm.LinearSVC(C=k, max_iter=100000)
    scores = cross_val_score(clf, x_train_transformed, y_train, cv=10, scoring='accuracy') #aplicamos cross validation
    cv_c_scores.append(scores.mean())
    

       

train_prec=[]
c_values2=np.logspace(-10,0, 10)

for k in c_values2 :
    
    clf = svm.LinearSVC(C=k, max_iter=100000).fit(x_train_transformed,y_train)
    train_prec.append(clf.score(x_train_transformed,y_train))


  
# Do the plotting
import matplotlib.pyplot as plt
plt.figure()

plt.semilogx(c_values2,train_prec,color='b' , marker='*' , label="train")
plt.semilogx(c_values,cv_c_scores,color='r' , marker='*' , label="cv")
locs, labels = plt.yticks()
plt.yticks(locs, list(map(lambda x: "%g" % x, locs)))
plt.ylabel('accuracy')
plt.xlabel('Parameter C')
plt.title('Gráfica ajuste de SVM')
plt.show()


#select the maximum because we are considering accuracy
c_opt=(np.array(c_values)[cv_c_scores.index(np.array(cv_c_scores).max())])
c_opt
#obtengo valor maximo que tengo que usar para mi clasificador(k)   

In [0]:
#entrenamos nuestro modelo
 clf = svm.LinearSVC(C=c_opt).fit(x_train_transformed,y_train)

In [0]:
from sklearn.metrics import classification_report

y_pred=clf.predict(x_test_transformed)
prestations=classification_report(y_test,y_pred)
print(prestations)
print('\n \n Matriz de confusion: ')
matriz= pd.crosstab(y_test,y_pred,rownames=['Real'], colnames=['Predicted'], margins= True)
matriz

In [0]:
def prestaciones(confusion):
  precision=(confusion[0][0]+confusion[1][1])/(confusion[0][0]+confusion[0][1]+confusion[1][0]+confusion[1][1])
  sensibilidad=(confusion[1][1])/(confusion[1][0]+confusion[1][1])
  especificidad=(confusion[0][0])/(confusion[0][0]+confusion[0][1])
  print('Especificidad: ', especificidad)
  print('Sensibilidad: ', sensibilidad)
  print('Precision: ', precision)

In [0]:
print(prestaciones(matriz))

In [0]:
coef= clf.coef_
print(coef)

SELECCIÓN DE CARACTERÍSTICAS

In [0]:
from sklearn import svm

def feature_plot(classifier, feature_names, top_features=24):
 coef = classifier.coef_.ravel()
 top_positive_coefficients = np.argsort(coef)[-top_features:]
 top_negative_coefficients = np.argsort(coef)[:top_features]
 top_coefficients = np.hstack([top_negative_coefficients, top_positive_coefficients])
 plt.figure(figsize=(25,10))
 colors = ['green' if c < 0 else 'blue' for c in coef[top_coefficients]]
 plt.bar(np.arange(2 * top_features), coef[top_coefficients], color=colors)
 feature_names = np.array(feature_names)
 plt.xticks(np.arange(1 + 2 * top_features), feature_names[top_coefficients], rotation=45, ha='right')
 plt.show()

#print(df.drop(['Class'], axis = 1).columns.values)

clf = svm.LinearSVC(C=c_opt).fit(x_train_transformed,y_train)
feature_plot(clf, df.drop(['Class'], axis = 1).columns.values)

TRABAJO CON CARACTERÍSTICAS SELECCIONADAS

In [0]:
x2 = df[['Cirrhosis','Iron','Alcohol','Creatinine','Encephalopathy','Grams_day','Obesity','Gender','MCV','PHT','Hemoglobin','Spleno','AFP','Hemochro','Hallmark','Endemic','Smoking','HBsAg','ALT','NASH','AHT']] 
y2=df['Class']
x_train, x_test, y_train, y_test = train_test_split(x2, y2,stratify=y, test_size=0.2, random_state=42)


In [0]:
#normalizamos datos
from sklearn import preprocessing

scaler=preprocessing.StandardScaler().fit(x_train)

x_train_transformed=scaler.transform(x_train)
x_test_transformed=scaler.transform(x_test)


In [0]:
clf = svm.LinearSVC(penalty='l2', C=c_opt, max_iter=100000).fit(x_train_transformed,y_train)

In [0]:
from sklearn.metrics import classification_report

y_pred=clf.predict(x_test_transformed)
prestations=classification_report(y_test,y_pred)
print(prestations)
print('\n \n Matriz de confusion: ')
matriz= pd.crosstab(y_test,y_pred,rownames=['Real'], colnames=['Predicted'], margins= True)
matriz

In [0]:
print(prestaciones(matriz))